In [7]:
import pandas as pd
import os
from genoslurm.genoslurm import chunks, GenCallJob

In [8]:
# cluster config variables
max_nodes = 99

# paths- MAKE SURE TO SET TO YOUR OWN HOME PATH IN THE CLUSTER
userhome = '/home/dan_datatecnica_com'
datapath = f'/genoslurm_nfs'
log_dir = f'{userhome}/logs'
ilmn_files_path = f'{userhome}/ilmn_files'
iaap = f'{userhome}/GenoTools/executables/iaap-cli-linux-x64-1.1.0-sha.80d7e5b3d9c1fdfc2e99b472a90652fd3848bbc7/iaap-cli/iaap-cli'

# copy these from gcs
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A2.bpm'
egt = f'{ilmn_files_path}/recluster_09272022.egt'

# paths for running pipelines
idat_path = f'{datapath}/idats'
plink_file_path = f'{datapath}/ped_bed'
# gcs_plink_path = f'gp2_uk/gp2_plink'
# gcs_idat_path = f'gp2_uk/gp2_idats'

In [9]:
# list of idats in directory of choice
!gsutil ls gs://gp2_uk/gp2_idats/ > cluster_scripts/idats.txt
idats_in = pd.read_csv('cluster_scripts/idats.txt', header=None)
idat_list = idats_in.loc[1:,0]
idat_list = [x.replace('gs://gp2_uk/gp2_idats',f'{datapath}/idats')[:-1] for x in idat_list]
# gcs_idat_paths = [x.replace('gs://', '').rstrip('/') for x in idats_in.loc[1:,0]]

# chunk list by max nodes
idat_list_chunks = chunks(idat_list, max_nodes)
# gcs_idat_paths_chunks = chunks(gcs_idat_paths, max_nodes)
# need this without gs://
# gcs_plink_path = 'gp2_uk/gp2_plink'

In [13]:
# the following is run in batches of <=99 jobs (1 job per sentrix barcode)
# sbatch_cmds saves the commands that will be run in the notebook using gcloud compute ...
sbatch_cmds = []

# loop through chunks of <=99 barcodes
for i, idat_list_chunk in enumerate(idat_list_chunks):

    input_file_path = f'inputs/call_gts_{i}.inputs'

    job_name = f'callgts_{i}'
    ntasks = len(idat_list_chunk)

    # this writes input_file path which contains 3 columns: 
    # 1. path to idat in the cluster 
    # 2. path to idat directory in gcs 
    # 3. path to output in gcs

    with open(input_file_path, 'w') as f:
        for j, idat in enumerate(idat_list_chunk):
            code = idat.split('/')[-1]
            # gcs_idat_path = f'{gcs_idat_path}/{code}'
            # outpath = f'{plink_file_path}/'
            # f.write(f"{idat}\t{gcs_idat_paths_chunks[i][j]}\t{gcs_idat_paths_chunks[i][j]}\n")
            f.write(f"{idat}\t{plink_file_path}\n")
    f.close()

    # now write command for this batch of <=99 sentrix barcodes
    sbatch_cmd = f"sbatch --job-name {job_name} --output={userhome}/logs/{job_name}.%A_%a.out --error={userhome}/logs/{job_name}.%A_%a.err --cpus-per-task=3 --time=05:00:00 --array=1-{ntasks} {userhome}/scripts/call_gts.sh {userhome}/{input_file_path}"
    sbatch_cmds.append(sbatch_cmd)
    print(sbatch_cmd)


sbatch --job-name callgts_0 --output=/home/dan_datatecnica_com/logs/callgts_0.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_0.%A_%a.err --cpus-per-task=3 --time=05:00:00 --array=1-99 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_0.inputs
sbatch --job-name callgts_1 --output=/home/dan_datatecnica_com/logs/callgts_1.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_1.%A_%a.err --cpus-per-task=3 --time=05:00:00 --array=1-99 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_1.inputs
sbatch --job-name callgts_2 --output=/home/dan_datatecnica_com/logs/callgts_2.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_2.%A_%a.err --cpus-per-task=3 --time=05:00:00 --array=1-17 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_2.inputs


In [14]:
# copy scripts to vm
!gcloud compute scp cluster_scripts/call_gts.sh genoslurm-uk-controller:{userhome}/scripts/ --project genotools --zone europe-west2-a
!gcloud compute scp cluster_scripts/call_gts.py genoslurm-uk-controller:{userhome}/scripts/ --project genotools --zone europe-west2-a
!gcloud compute scp inputs/* genoslurm-uk-controller:{userhome}/inputs/ --project genotools --zone europe-west2-a

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
call_gts.sh                                   100%  348     2.9KB/s   00:00    
/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
call_gts.py                                   100% 1597    12.8KB/s   00:00    
/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
call_gts_0.inputs                             100% 5643    44.3KB/s   00:00    
call_gts_1.inputs                             100% 5643    43.9KB/s   00:00    
call_gts_2.inputs                             100%  969     7.7KB/s   00:00    


In [15]:
# launch commands
for cmd in sbatch_cmds:
    !gcloud compute ssh --zone europe-west2-a --project genotools genoslurm-uk-controller --command "{cmd}"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
Submitted batch job 1722
/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
Submitted batch job 1821
/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
Submitted batch job 1920


In [21]:
sbatch_cmds

['sbatch --job-name callgts_0 --output=/home/dan_datatecnica_com/logs/callgts_0.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_0.%A_%a.err --cpus-per-task=3 --time=01:00:00 --array=1-99 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_0.inputs',
 'sbatch --job-name callgts_1 --output=/home/dan_datatecnica_com/logs/callgts_1.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_1.%A_%a.err --cpus-per-task=3 --time=01:00:00 --array=1-99 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_1.inputs',
 'sbatch --job-name callgts_2 --output=/home/dan_datatecnica_com/logs/callgts_2.%A_%a.out --error=/home/dan_datatecnica_com/logs/callgts_2.%A_%a.err --cpus-per-task=3 --time=01:00:00 --array=1-17 /home/dan_datatecnica_com/scripts/call_gts.sh /home/dan_datatecnica_com/inputs/call_gts_2.inputs']